<a href="https://colab.research.google.com/github/IvaroEkel/Probabilistic-Machine-Learning_Lecture/blob/main/Linear_Regression_1_Frequentist_Bayesian_OLS_MLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression: Frequentist - Bayesian | OLS - MLS

In this notebook, we explore linear regression from both:
- **Frequentist**: Point estimates via Maximum Likelihood.
- **Bayesian**: Prior, posterior, and predictive distribution.

and

- **Ordinary Linear Regression** (OLS) (one variable)
- **Multiple Linear Regression** (MLS) (multiple variables)


# Linear Regression on an example of ML fairness: the Boston Housing Dataset

In [ ]:
!pip install kaggle

In [ ]:

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import pymc as pm
import arviz as az


In [ ]:
from sklearn.datasets import load_boston

In [ ]:

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    # data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :3]])
    target = raw_df.values[1::2, 2]


 The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
 prices and the demand for clean air', J. Environ. Economics & Management,
 vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
 ...', Wiley, 1980.   N.B. Various transformations are used in the table on
 pages 244-261 of the latter.

**Variables in order**:
  - CRIM     per capita crime rate by town
  - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
  - INDUS    proportion of non-retail business acres per town
  - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
  - NOX      nitric oxides concentration (parts per 10 million)
  - RM       average number of rooms per dwelling
  - AGE      proportion of owner-occupied units built prior to 1940
  - DIS      weighted distances to five Boston employment centres
  - RAD      index of accessibility to radial highways
  - TAX      full-value property-tax rate per $10,000$ usd
  - PTRATIO  pupil teacher ratio by town
  - B        $1000(B_k - 0.63)^2$ where $B_k$ is the proportion of blacks by town
  - LSTAT    $%$ lower status of the population
  - MEDV     Median value of owner-occupied homes in $1000's

In [ ]:
new_col_names = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
    'PTRATIO', 'B', 'LSTAT', 'MEDV'
]
len(new_col_names)


In [ ]:
df = pd.DataFrame(data,columns= new_col_names)
df

## FREQUENTIST REGRESSION: MEDV vs RM ---

In [ ]:
X = df[['RM']]
y = df['MEDV']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:

# Plot
sns.regplot(x='RM', y='MEDV', data=df)
plt.title('Linear Regression: MEDV vs RM')
plt.show()


## FREQUENTIST MULTIPLE REGRESSION ---

In [ ]:
y = df['MEDV']
X = df.drop('MEDV', axis=1)
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

# --- BAYESIAN SINGLE REGRESSION: MEDV vs RM ---

In [ ]:
X = df[['RM']].values.flatten()
y = df['MEDV'].values

with pm.Model() as model_bayes_single:
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=5)
    mu = alpha + beta * X
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

az.plot_trace(trace)
plt.show()


# --- BAYESIAN MULTIPLE REGRESSION ---

In [ ]:
X = df.drop('MEDV', axis=1)
X_scaled = StandardScaler().fit_transform(X)
y = df['MEDV'].values

with pm.Model() as model_bayes_multi:
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    betas = pm.Normal('betas', mu=0, sigma=1, shape=X.shape[1])
    sigma = pm.HalfNormal('sigma', sigma=5)
    mu = alpha + pm.math.dot(X_scaled, betas)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    trace_multi = pm.sample(1000, tune=1000, return_inferencedata=True)

az.plot_trace(trace_multi, var_names=['alpha', 'betas'])
plt.show()

# Posterior predictive checks
with model_bayes_multi:
    ppc = pm.sample_posterior_predictive(trace_multi, var_names=['y_obs'])
# az.plot_ppc(az.from_pymc3(posterior_predictive=ppc, model=model_bayes_multi))
# plt.show()
# Remove the conversion using from_pymc3 since it's not needed.
# Replace line 19 with the following:
az.plot_ppc(trace_multi, group="posterior_predictive", var_names=['y_obs'])